K-Nearest Neighbours Algorithm

In [141]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import math

import operator

In [142]:
from sklearn.preprocessing import LabelEncoder

# Read dataset
df = pd.read_csv('../data/BCCC-CIRA-CIC-DoHBrw-2020.csv')
""" df['Label'] = df['Label'].map({
    'Malicious': 1,
    'Benign': 2
})  # Label values - 1 for Malignant and 2 for Benign """
labels = df['Label'].tolist()
df['Class'] = labels  #Copying values of Label to newly created labels column


# Check for null values
#print(df.isnull().sum())

# Fill null values with mean
if df.isnull().sum().sum() > 0:
    df = df.fillna(df.mean())

# Encode categorical data
encoder = LabelEncoder()
df['Label'] = encoder.fit_transform(df['Label'])


df = df.drop(['Label'],
             axis=1)  #Dropping unncessary columns

sample_size = int(len(df) / 40)  # Specify the desired sample size
print("Sample size: " + str(sample_size))
df_full = df
df = df_full.sample(n=sample_size, random_state=13)  # Randomly sample 'sample_size' number of rows

print(encoder.classes_)

df.describe()
#df.head()  #Displaying first five rows of the dataset

Sample size: 12477
['Benign' 'Malicious']


,FlowBytesSent,FlowSentRate,FlowBytesReceived,FlowReceivedRate,PacketLengthVariance,PacketLengthStandardDeviation,PacketLengthMean,PacketLengthMedian,PacketLengthMode,PacketLengthSkewFromMedian,...,PacketTimeSkewFromMode,PacketTimeCoefficientofVariation,ResponseTimeTimeVariance,ResponseTimeTimeStandardDeviation,ResponseTimeTimeMean,ResponseTimeTimeMedian,ResponseTimeTimeMode,ResponseTimeTimeSkewFromMedian,ResponseTimeTimeSkewFromMode,ResponseTimeTimeCoefficientofVariation
count,1.247700e+04,1.247700e+04,1.247700e+04,1.247700e+04,1.247700e+04,12477.000000,12477.000000,12477.000000,12477.000000,12477.000000,...,12477.000000,12477.000000,12477.000000,12477.000000,12477.000000,12477.000000,12477.000000,12477.000000,12477.000000,12477.000000
mean,4.030295e+04,4.719079e+04,4.290608e+04,3.169948e+04,9.341516e+04,220.195996,173.046697,94.969116,70.705057,0.452964,...,1.238405,0.976253,1.609498,0.309404,0.450321,0.402333,0.222478,-1.006847,-0.072998,1.103353
std,1.582774e+05,4.184716e+05,1.418057e+05,2.566129e+05,1.566658e+05,211.967984,86.770260,32.330563,14.579278,1.536554,...,0.719776,0.519682,10.194108,1.205040,2.116633,2.374612,1.800563,3.188285,3.234884,1.869352
min,5.500000e+01,1.488526e+00,5.400000e+01,1.576258e+00,0.000000e+00,0.000000,56.000000,54.000000,54.000000,-10.000000,...,-4.072893,0.101544,0.000000,0.000000,0.000007,0.000003,0.000000,-10.000000,-10.000000,0.000000
25%,6.180000e+02,5.414777e+01,4.580000e+02,1.417723e+02,4.692099e+02,21.661253,92.000000,76.000000,66.000000,0.211341,...,0.627944,0.578279,0.000017,0.003982,0.010589,0.012514,0.000016,-1.816621,0.410334,0.533672
50%,1.807000e+03,3.677016e+02,4.827000e+03,4.353435e+02,1.797543e+04,134.070249,151.912897,87.000000,68.000000,0.986397,...,1.224944,0.757289,0.000077,0.008710,0.015144,0.015410,0.000034,-0.032925,0.926644,0.797324
75%,5.363000e+03,3.631044e+03,7.764000e+03,4.209092e+03,1.418550e+05,376.635599,228.758621,105.000000,68.000000,1.201609,...,1.716543,1.518324,0.000323,0.017764,0.024946,0.016583,0.015105,0.903152,1.307640,1.186038
max,6.209568e+06,6.176471e+06,4.293889e+06,3.882353e+06,1.369730e+06,1170.354496,586.833333,287.000000,213.000000,2.879882,...,9.847905,4.171592,144.281231,11.972824,28.017596,28.017596,28.017596,2.770685,3.558791,58.760508


Train-Test Split

In [143]:
np.random.seed(1)
msk = np.random.rand(
    len(df)) < 0.7  #An array containing True(with probability 0.7) and False
train = df[msk]  #Rows having array value true
test = df[~msk]  #Rows having array value False
print('Number of observations in the training data: ', len(train))
print('Number of observations in the test data: ', len(test))

Number of observations in the training data:  8758
Number of observations in the test data:  3719


In [144]:
train.head()

,FlowBytesSent,FlowSentRate,FlowBytesReceived,FlowReceivedRate,PacketLengthVariance,PacketLengthStandardDeviation,PacketLengthMean,PacketLengthMedian,PacketLengthMode,PacketLengthSkewFromMedian,...,PacketTimeCoefficientofVariation,ResponseTimeTimeVariance,ResponseTimeTimeStandardDeviation,ResponseTimeTimeMean,ResponseTimeTimeMedian,ResponseTimeTimeMode,ResponseTimeTimeSkewFromMedian,ResponseTimeTimeSkewFromMode,ResponseTimeTimeCoefficientofVariation,Class
44565,171,2579.556192,171,2579.556192,380.250000,19.500000,85.500000,85.5,66,0.000000,...,0.814335,0.000000e+00,0.000000,0.025227,0.025227,0.025227,-10.000000,-10.000000,0.000000,Benign
82229,1085,2732.605142,4099,10323.454819,215000.869806,463.681863,272.842105,68.0,68,1.325319,...,0.807566,9.919523e-05,0.009960,0.009760,0.007710,0.000003,0.617590,0.979633,1.020476,Malicious
63378,110,2.441408,132,2.929690,30.250000,5.500000,60.500000,60.5,55,0.000000,...,0.999444,1.066619e-09,0.000032,0.025048,0.025048,0.025015,0.000000,1.000000,0.001295,Benign
238800,293125,2496.034998,169118,1440.083400,14389.935956,119.958059,166.095221,68.0,68,2.453238,...,0.582091,3.346842e-10,0.000018,0.000036,0.000038,0.000038,-0.361898,-0.120633,0.511115,Malicious
395083,353,82.875157,327,76.771038,487.750000,22.085063,85.000000,78.0,66,0.950869,...,0.577349,1.446222e-09,0.000038,0.000053,0.000032,0.000020,1.630325,0.858988,0.718952,Benign


In [145]:
test.head()

,FlowBytesSent,FlowSentRate,FlowBytesReceived,FlowReceivedRate,PacketLengthVariance,PacketLengthStandardDeviation,PacketLengthMean,PacketLengthMedian,PacketLengthMode,PacketLengthSkewFromMedian,...,PacketTimeCoefficientofVariation,ResponseTimeTimeVariance,ResponseTimeTimeStandardDeviation,ResponseTimeTimeMean,ResponseTimeTimeMedian,ResponseTimeTimeMode,ResponseTimeTimeSkewFromMedian,ResponseTimeTimeSkewFromMode,ResponseTimeTimeCoefficientofVariation,Class
419667,21253,1791.528294,20084,1680.815713,21633.071300,147.080476,154.081819,114.0,115,0.817630,...,0.642373,1.120331e-03,0.030383,0.020175,0.014658,0.000048,0.589664,0.792956,1.495806,Benign
185640,171,10953.814618,105,6726.026520,338.000000,18.384776,92.000000,105.0,105,-2.121320,...,0.707107,0.000000e+00,0.000000,0.015591,0.015591,0.015591,-10.000000,-10.000000,0.000000,Benign
301917,16073,179.892071,21709,242.971254,38685.169763,196.685459,145.315385,84.0,68,0.935230,...,0.709361,1.295619e-04,0.011383,0.018891,0.015570,0.000015,0.875285,1.658330,0.602538,Malicious
4113,342,5.794081,276,4.675925,372.489796,19.299995,88.285714,105.0,105,-2.598076,...,1.153373,1.684580e-09,0.000041,0.025346,0.025346,0.025305,0.000000,1.000000,0.001616,Benign
421594,1739,51.968803,4829,144.311300,151755.459184,389.558031,234.571429,76.0,68,1.221164,...,1.564683,6.871375e-05,0.008289,0.011559,0.016146,0.000017,-1.659916,1.392438,0.717108,Malicious


Euclidean distance

In [146]:
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

Calculating the Nearest Neighbors

In [147]:
def getNeighbors(trainingSet, testInstance, k):
    distances = []  #List to store all the distance values
    length = len(testInstance) - 1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)  #Calculating the Euclidean Distance
        distances.append((trainingSet[x],dist))  #Appending the distance values to the 'distance' list
    distances.sort(key=operator.itemgetter(1))  #Sorting based on the disance value
    neighbors = []  #List to store all the neighbors
    for x in range(k):
        neighbors.append(distances[x][0])  #Number of neighbors is dependent on the value of k
    return neighbors

In [148]:
def getResponse(neighbors):
    classVotes = {}  #Dictionary to store labels with their counts
    for x in range(len(neighbors)):
        response = neighbors[x][-1]  #Label value of the neighbors
        if response in classVotes:
            classVotes[response] += 1  #if label value is already present increment it by 1
        else:
            classVotes[response] = 1  #If the label value is not yet present add it to the dictionary
    sortedVotes = sorted(
        classVotes.items(), key=operator.itemgetter(1), reverse=True
    )  #Sort the dictinary based on the count value in descending order
    return sortedVotes[0][0]  #Return the label with highest number of occurences

Calculating the accuracy

In [149]:
def getAccuracy(testSet, predictions):
    correct = 0  #Variable to store the correct predictions
    for x in range(len(testSet)):
        if testSet[x][-1] is predictions[x]:  #Checking whether the predicted value is same as label value
            correct += 1  #Incremented when both values are same
    return (correct / float(len(testSet))) * 100.0  #Accuracy = No. of Correct pred / Total number of pred

Implementation

In [150]:
predictions = []  #List to store the predicted values
k = 3  # 3-Nearest Neighbors
trainingSet = train.values.tolist()  #List containing training data
testSet = test.values.tolist()  #List containing test data
for x in range(len(testSet)):
    neighbors = getNeighbors(trainingSet, testSet[x], k)
    result = getResponse(neighbors)
    predictions.append(result)  # Storing the predicted values
    # print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))

Confusion Matrix

In [151]:
print("Confusion Matrix")
y_test = []
for i in testSet:
    y_test.append(i[-1])
from sklearn.metrics import confusion_matrix, accuracy_score
res = confusion_matrix(y_test, predictions)
print(res)

Confusion Matrix
[[1840   76]
 [  58 1745]]


Accuracy

In [152]:
accuracy = accuracy_score(y_test, predictions) * 100
print('Accuracy: ' + repr(accuracy) + '%')

Accuracy: 96.39688088195751%
